In [1]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/data_collection')

import bonds_parser
importlib.reload(bonds_parser)
from bonds_parser import MOEXBondHistoricalParser, Logger


/Users/aeshef/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ImportError: cannot import name 'Logger' from 'bonds_parser' (/Users/aeshef/Documents/GitHub/kursach/pys/data_collection/bonds_parser.py)

In [ ]:
like_print_log = Logger(name="main", format="%(message)s", store=True)

bond_parser = MOEXBondHistoricalParser(log=like_print_log, verbose=False)

result = bond_parser.parse_from_2024_to_now(
    include_non_trading_days=True,  # Включаем выходные дни
    analyze_continuity=True,        # Анализируем непрерывность и создаем полный датасет
    yield_range=(0, 100),           # Широкий диапазон - берем все
    price_range=(0, 100),           # Широкий диапазон - берем все
    duration_range=(0, 7200)         # Широкий диапазон - берем все
)

print(f"Всего дней с данными: {len(result)}")
trading_days = sum(1 for date in result.keys() if datetime.strptime(date, '%Y-%m-%d').weekday() < 5)
print(f"Торговых дней: {trading_days}")
print(f"Нерабочих дней с данными: {len(result) - trading_days}")



Запуск массового парсинга данных с 2024-01-01 по 2025-04-23
Запрошен период с 2024-01-01 по 2025-04-23
Получение облигаций за период с 2024-01-01 по 2025-04-23
Обработка даты: 2024-01-01 (рабочий день)
Получение списка облигаций на 2024-01-01


KeyboardInterrupt: 

In [ ]:
optimal_threshold = bond_parser.find_optimal_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    min_bonds=20,
    max_threshold=99
)

print(f"Оптимальный порог непрерывности: {optimal_threshold}%")

dataset = bond_parser.generate_dataset_with_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    continuity_threshold=optimal_threshold
)

if dataset is not None:
    print(f"Создан датасет с {len(dataset['secid'].unique())} облигациями и {len(dataset['date'].unique())} датами")
else:
    print("Не удалось создать датасет")

Поиск оптимального порога непрерывности для получения минимум 20 облигаций
Порог 99%: 49 облигаций
Найден оптимальный порог: 99% с 49 облигациями
Оптимальный порог непрерывности: 99%
Генерация датасета с порогом непрерывности 99% с 2024-01-01 по 2025-04-22
Найдено 49 облигаций с непрерывностью >= 99%
Датасет с порогом непрерывности 99% создан и сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/threshold_dataset_20240101_20250422_99
Всего дат: 340, облигаций: 49
Создан датасет с 36 облигациями и 340 датами


/Users/aeshef/Documents/GitHub/kursach/pys/data_collection/bonds_parser.py:1063: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_data_filled = price_data.ffill()
/Users/aeshef/Documents/GitHub/kursach/pys/data_collection/bonds_parser.py:1064: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yield_data_filled = yield_data.ffill()
